In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import rc

font = {'family':'sans-serif', 'size'   : 18}

rc('font', **font)

In [7]:
def make_unit(value, param_dict, param):
    unit_value = (value - param_dict[param][1])/(param_dict[param][2] - param_dict[param][1])
    return unit_value

def params_unit(value, param_dict):
    unit_params = np.empty((6))
    for i,key in enumerate(['qv_bl','inv','delt','delq','na','baut']):  # needs to be this order
        unit_value = make_unit(value[i], param_dict, key)
        unit_params[i] = unit_value
    return unit_params

def design_unit(design, param_dict):
    unit_design = np.empty((np.shape(design)))
    for j,member in enumerate(design):
        unit_params = params_unit(member, param_dict)
        unit_design[j] = unit_params
    return unit_design
    
def plot_perturbation(key, unit_values, a_quarts):
    fig, ax = plt.subplots(figsize=(8,6))
    
    ax.plot(unit_names, [0.25]*len(unit_names),alpha=a_quarts, color="black")
    ax.plot(unit_names, [0.5]*len(unit_names),alpha=a_quarts, color="black")
    ax.plot(unit_names, [0.75]*len(unit_names),alpha=a_quarts, color="black")
    dots = []
    for name, values, col in zip(unit_names, unit_values, ['C0', 'C1', 'C2', 'C3','C4','C5']):
        dot = ax.scatter(name, values, s=250, c=col)
        dots.append(dot)
    ax.set_xlabel("Parameters")
    ax.set_ylabel("Unit value")
    ax.set_yticks([0,0.25,0.5,0.75,1])
    ax.set_ylim((0,1))
    ax.set_title(key)
    
    lgd = ax.legend(dots, ["$7 < BL~q_{v} < 11$", "$500 < Inversion < 1300$", "$2 < \Delta~\theta < 21$", "$-7 < \Delta~q_{v} < -11$", 
                                "$10 < N_{a} < 500$", "$-2.3 < b_{aut} < 1.3$"], loc = (1.05, 0.45))
    plt.rc('xtick', labelsize='large')
    plt.rc('ytick', labelsize='medium')
    fig.savefig(f"lh_design/unit_value_plots/{key}_unit_values.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
    #plt.show()
    plt.close()
    return None

def main(path, design_name, param_dict):
    design = np.loadtxt(path, delimiter=',', skiprows=1)
    unit_values = design_unit(design, param_dict)
    
    for i,row in enumerate(unit_values):
        plot_perturbation(f'{design_name}{i}', row, 0.2)
        
    return None

In [8]:
names=['$BL~q_{v}~(g~kg^{-3})$', '$Inversion~(m)$', r'$\Delta~\theta~(K)$', '$\Delta~q_{v}~(g~kg^{-3})$', '$BL~accum.~(cm^{-3})$', '$b_{aut}~parameter$']
unit_names_long=['$BL~q_{v}$', '$Inversion$', r'$\Delta~\theta$', '$\Delta~q_{v}$', '$BL~accum.$', '$b_{aut}~param$']
unit_names=['$BL~q_{v}$', '$Inv$', r'$\Delta~\theta$', '$\Delta~q_{v}$', '$N_{a}$', '$b_{aut}$']

param_dict={'qv_bl': [unit_names[0], 7, 11], 'inv': [unit_names[1], 500, 1300], 'delt': [unit_names[2], 2, 21], 
            'delq': [unit_names[3], -7, -1], 'na': [unit_names[4], 10, 500], 'baut': [unit_names[5], -2.3, -1.3]}
    
#main("/home/users/eers/sct/lh_design/SCT_EmulatorInputsDesign.csv", "em", param_dict)
#main("/home/users/eers/sct/lh_design/SCT_ValidationInputsDesign.csv", "val", param_dict)
main("/home/users/eers/sct/lh_design/SCT_EmulatorOAT.csv", "oat", param_dict)

In [12]:
em_design

,$BL~q_{v}~(g~kg^{-3})$,$Inversion~(m)$,$\Delta~\theta~(K)$,$\Delta~q_{v}~(g~kg^{-3})$,$BL~accum.~(cm^{-3})$,$b_{aut}~parameter$
0,10.461300,900.000000,11.157000,-6.342200,150.000000,-1.790000
1,7.409852,632.601449,7.738419,-3.450394,320.554774,-1.680355
2,7.660220,663.222733,13.086274,-4.466174,57.110258,-1.349428
3,9.875011,505.938876,12.924094,-1.851715,286.747888,-1.932485
4,10.519952,892.374904,4.853165,-2.145602,461.133661,-2.231373
...,...,...,...,...,...,...
56,10.561848,1010.799022,18.130205,-1.717622,337.357244,-1.400085
57,7.274798,719.913686,3.106220,-3.909289,469.153719,-2.292228
58,7.365513,857.583572,17.702907,-6.755257,239.720767,-1.362997
59,7.099124,523.989290,19.402004,-1.393869,112.650229,-2.126616
